In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import sklearn
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize.casual import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
from nltk.corpus import stopwords
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score

In [2]:
stop_words = set(stopwords.words('english'))
data = pd.read_csv('data/train.csv')

In [3]:
def get_hashtag_column(dataframe):
    hashtags = []
    for text in dataframe.text:
        result = re.findall('#\w+', text)
        if result != []:
            result = [w[1:].lower() for w in result]
            hashtags.append(' '.join(result))
    return hashtags

In [4]:
def lemmatize_texts(texts):
    lemmatizer = WordNetLemmatizer()
    result = []
    for t in texts:
        lemmatized_words = []
        t = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                  'url', t)
        #t = re.sub('\!+', '', t)
        #t = re.sub('\?+', '', t)
        #t = re.sub('\d+[\:|\.]?\d*\s')
        t = re.sub('\d+', '', t)
        tokens = re.findall('''\d+,\d+|\w+'\w+|#?\w+-?\w+|\w+\*+\w+''', t)
        
        if tokens == []:
            print(t)
        
        for token in tokens:
            if token.lower() not in stop_words:
                lemmatized_words.append(lemmatizer.lemmatize(token).lower())
        result.append(' '.join(lemmatized_words).replace('#', ''))
    return result

In [5]:
def tokenizer(text):
    return text.split(' ')

In [6]:
all_lemmatized_texts = lemmatize_texts(data.text)
all_lemmatized_tokens = [w for t in all_lemmatized_texts for w in t.split(' ')]
print('Total words: ', len(all_lemmatized_tokens))
print('Unique_words: ', len(set(all_lemmatized_tokens)))

Total words:  72070
Unique_words:  15827


In [7]:
# Most common words in dataset
freq = nltk.probability.FreqDist(all_lemmatized_tokens)
freq.most_common(20)

[('url', 4114),
 ('û_', 349),
 ('like', 347),
 ('fire', 339),
 ('amp', 327),
 ("i'm", 248),
 ('get', 248),
 ('new', 217),
 ('one', 201),
 ('news', 199),
 ('people', 198),
 ('disaster', 159),
 ('video', 158),
 ('emergency', 156),
 ('time', 147),
 ('body', 145),
 ('day', 141),
 ('police', 141),
 ('year', 133),
 ('would', 132)]

In [8]:
# Most common words in real tweets
real_tweets = data[data.target == 1].text
real_tweets = lemmatize_texts(real_tweets)
freq_real = nltk.probability.FreqDist([w for t in real_tweets for w in t.split(' ')])
freq_real.most_common(10)

[('url', 2228),
 ('fire', 253),
 ('û_', 172),
 ('news', 142),
 ('amp', 130),
 ('disaster', 120),
 ('california', 110),
 ('suicide', 109),
 ('police', 107),
 ('people', 106)]

In [9]:
# Most common words in fake tweets
fake_tweets = data[data.target == 0].text
fake_tweets = lemmatize_texts(fake_tweets)
freq_fake = nltk.probability.FreqDist([w for t in fake_tweets for w in t.split(' ')])
freq_fake.most_common(10)

[('url', 1886),
 ('like', 254),
 ("i'm", 203),
 ('amp', 197),
 ('get', 181),
 ('û_', 177),
 ('new', 163),
 ('one', 132),
 ('body', 114),
 ('would', 98)]

In [10]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(all_lemmatized_texts, 
                                                    data.target, test_size=0.2, random_state=42)

In [11]:
# Vectorize texts
vectorizer = CountVectorizer(ngram_range=(1,2), tokenizer=tokenizer)
train = vectorizer.fit_transform(X_train)
test = vectorizer.transform(X_test)

## LinearSVC

In [65]:
# Train SVM (accuracy: 0.78)
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=0)
# svc_parameters = {'tol' : [1e-7, 1e-6, 1e-5], 
#                   'max_iter': np.arange(400, 1000, 200)
#                  }
# grids_svc = GridSearchCV(svc, svc_parameters, n_jobs=-1, cv=5)
svc.fit(train, y_train)
# clf.score(test, y_test)

In [66]:
# grids_clf.fit(train, y_train)

c:\users\svetl\for project\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=0, tol=0.0001, verbose=0),
             iid='warn', n_jobs=-1,
             param_grid={'max_iter': array([1000, 2000]),
                         'tol': array([5.0e-07, 1.0e-06, 1.5e-06, 2.0e-06, 2.5e-06, 3.0e-06, 3.5e-06,
       4.0e-06, 4.5e-06, 5.0e-06, 5.5e-06, 6.0e-06, 6.5e-06, 7.0e-06,
       7.5e-06, 8.0e-06, 8.5e-06, 9.0e-06, 9.5e-06, 1.0e-05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [67]:
# grids_clf.best_score_, grids_clf.best_params_

(0.7829228243021347, {'max_iter': 1000, 'tol': 5e-07})

## RandomForestClassifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state=42, 
                                n_estimators=71, 
                                min_samples_leaf=2, 
                                max_features=300, 
                                oob_score=True)
forest.fit(train, y_train)
# forest_params = {'max_features': np.arange(100, 1000, 100)}
# grid_forest = GridSearchCV(forest, forest_params, n_jobs=-1, cv=5)
print(forest.oob_score_)

0.8003284072249589


In [ ]:
# grid_forest.fit(train, y_train)

In [ ]:
# grid_forest.best_score_, grid_forest.best_params_

In [ ]:
# (0.7779967159277504, {'n_estimators': 71})
# (0.790311986863711, {'max_features': 'auto'})
# (0.7957307060755336, {'max_features': 300})

## AdaBoostClassifier

In [110]:
from sklearn.ensemble import AdaBoostClassifier
boost = AdaBoostClassifier(base_estimator=sklearn.tree.DecisionTreeClassifier(max_depth=10),
                           random_state=42, algorithm='SAMME')
boost_params = {'n_estimators': np.arange(1, 10),
                'learning_rate': np.arange(0.01, 0.1, 0.01)
                }
grid_boost = GridSearchCV(boost, boost_params, n_jobs=-1, cv=5)
# boost.fit(train, y_train)

In [111]:
grid_boost.fit(train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME',
                                          base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                                criterion='gini',
                                                                                max_depth=10,
                                                                                max_features=None,
                                                                                max_leaf_nodes=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                                                                                min_samples_leaf=1,
                                                                                min_

In [112]:
grid_boost.best_score_, grid_boost.best_params_

(0.6609195402298851, {'learning_rate': 0.08, 'n_estimators': 9})

# Neural Network

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device('cuda:0')
    from torch.cuda import FloatTensor
else:
    device = torch.device('cpu')
    from torch import FloatTensor
    
try:
    from google.colab import drive
    is_in_colab = True
except:
    is_in_colab = False
    

## LinearNN

In [23]:
# fit settings
batch_size = 50
epoch_count = 10

# optim settings
learning_rate = 1e-3
weight_decay = 0

# model settings
linear1_out = int(train.shape[1]**0.5)
output = 2

# scheduler settings
factor = 0.1
patience = 2
threshold = 1e-2

model = nn.Sequential(nn.Linear(train.shape[1], linear1_out),
                      nn.ReLU(inplace=True),
                      nn.Linear(linear1_out, output),
                      nn.ReLU(inplace=True)
                     ).to(device)

loss_func = nn.CrossEntropyLoss()

optimizer = optim.Adam(
                        model.parameters(),
                        lr=learning_rate, 
                        weight_decay=weight_decay
                    )

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=factor, 
                              patience=patience, verbose=True, threshold=threshold
                              )